# 2. Data Preprocessing

## Setup environment

In [27]:
import numpy as np
import pandas as pd
import re

## Data Preprocessing

In [28]:
df = pd.read_csv('../../data/raw/data.csv')

In [29]:
df.head()

,name,genre,tomatometer_score,tomatometer_count,audience_score,audience_count,classification,runtime,release_year,original_language,url
0,A Castle for Christmas,"Holiday, Romance, Comedy",74%,23,40%,100+,NaN,1h 38m,2021,English,https://www.rottentomatoes.com/m/a_castle_for_...
1,Pinocchio,"Kids & family, Fantasy, Animation",100%,61,73%,"250,000+",G,1h 27m,1940,English,https://www.rottentomatoes.com/m/pinocchio_1940
2,The Informer,"Mystery & thriller, Crime, Drama",64%,58,60%,250+,R (Strong Violence|Pervasive Language),1h 53m,2019,English,https://www.rottentomatoes.com/m/the_informer_...
3,They Cloned Tyrone,"Sci-fi, Comedy",95%,129,100%,Fewer,R (Violence|Drug Use|Some Sexual Material|Perv...,2h 2m,2023,English,https://www.rottentomatoes.com/m/they_cloned_t...
4,1917,"War, History, Drama",89%,472,88%,"25,000+",R (Some Disturbing Images|Language|Violence),1h 59m,2019,English,https://www.rottentomatoes.com/m/1917_2019


We can see that there is a URL column which is not useful for our analysis. We can drop that column.

In [30]:
df = df.drop(axis = 1, columns = 'url')

In [31]:
df.head()

,name,genre,tomatometer_score,tomatometer_count,audience_score,audience_count,classification,runtime,release_year,original_language
0,A Castle for Christmas,"Holiday, Romance, Comedy",74%,23,40%,100+,NaN,1h 38m,2021,English
1,Pinocchio,"Kids & family, Fantasy, Animation",100%,61,73%,"250,000+",G,1h 27m,1940,English
2,The Informer,"Mystery & thriller, Crime, Drama",64%,58,60%,250+,R (Strong Violence|Pervasive Language),1h 53m,2019,English
3,They Cloned Tyrone,"Sci-fi, Comedy",95%,129,100%,Fewer,R (Violence|Drug Use|Some Sexual Material|Perv...,2h 2m,2023,English
4,1917,"War, History, Drama",89%,472,88%,"25,000+",R (Some Disturbing Images|Language|Violence),1h 59m,2019,English


### How many rows and how many columns does the raw data have? (0.25 points)

In [32]:
shape = df.shape

In [33]:
print(f'Number of rows: {shape[0]}')
print(f'Number of columns: {shape[1]}')

Number of rows: 1215
Number of columns: 10


### What does each line mean?

Each line in the dataset represents a movie/ series information, such as title, release year, genre, etc.

### Does the dataset have duplicated rows? 

In [34]:
unique_rows = df.drop_duplicates().shape[0] - shape[0]
print(f'Number of duplicate rows: {unique_rows}')

Number of duplicate rows: 0


### What does each column mean?

| Column Name | Description |
| --- | --- |
| Name | Name of the movie/ series |
| Genre | Genre of the movie/ series |
| Tomatometer Score | Tomatometer Score of the movie/ series |
| Tomatometer Count | Tomatometer Count of the movie/ series |
| Audience Score | Audience Score of the movie/ series |
| Audience Count | Audience Count of the movie/ series |
| Classification | The movie's age suitability rating |
| Runtime | Runtime of the movie/ series |
| Release Year | Release Year of the movie/ series |
| Original Language | Original Language of the movie/ series |

### Data type of each column

In [35]:
dtypes = df.dtypes

In [36]:
dtypes

name                 object
genre                object
tomatometer_score    object
tomatometer_count    object
audience_score       object
audience_count       object
classification       object
runtime              object
release_year          int64
original_language    object
dtype: object

Here, release year should be a string instead of an integer.

In [37]:
df['release_year'] = df['release_year'].astype(str)

In the Genre column, there are multiple genres for each movie/ series. We should pick the first genre as the main genre of the movie/ series.
The same goes for the Classification column.

In [38]:
df['genre'] = df['genre'].str.split(',').str[0]
df['classification'] = df['classification'].str.split(' ').str[0]

### Missing ratios of categorical columns

In [39]:
# YOUR CODE HERE
df_copy = df.copy()
df_copy = df_copy.drop(axis = 1, columns=['tomatometer_score', 'tomatometer_count', 'audience_score', 'audience_count', 'runtime', 'release_year'])
def missing_ratio(s):
    # raise NotImplementedError()
    return (s.isna().mean() * 100).round(1)

def num_values(s):
    # raise NotImplementedError()
    s = s.str.split(';')
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    # raise NotImplementedError()
    s = s.str.split(';')
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

cat_col_info_df = df_copy.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,name,genre,classification,original_language
missing_ratio,0.0,0.0,19.0,0.0
num_values,1207,21,8,27
value_ratios,"{'Risen': 0.2, 'Pinocchio': 0.2, 'Halloween': ...","{'Kids & family': 12.8, 'Comedy': 8.8, 'Action...","{'R': 44.4, 'PG-13': 31.5, 'PG': 17.8, 'G': 4....","{'English': 89.0, 'Japanese': 4.0, 'English (U..."


We can see that the classification column's missing ratio is 19, which is not too high. We can fill the missing values with a new category called "Not Rated".

In [40]:
df['classification'] = df['classification'].fillna('Not Rated')

```tomatometer_score, tomatometer_count, audience_score, audience_count``` and ```runtime``` should be **numeric**.  

- We'll convert ```tomatometer_score``` and ```audience_score``` to its float equivalence.

In [41]:
df['tomatometer_score'] = df['tomatometer_score'].str.strip('%')
df['audience_score'] = df['audience_score'].str.strip('%')

#Convert '--' value to 0
df.loc[df['tomatometer_score'] == '--', 'tomatometer_score'] = np.nan
df.loc[df['audience_score'] == '--', 'audience_score'] = np.nan

df[['tomatometer_score','audience_score']] = df[['tomatometer_score','audience_score']].astype(float) / 100

- ```audience_count``` and ```tomatometer_count``` has value 'fewer'.
- To determine value of 'fewer', we'll find the numerical min of each columns.

In [42]:
#Remove all symbols
df['audience_count'] = df['audience_count'].str.replace(',','').str.replace('+','')
df['tomatometer_count'] = df['tomatometer_count'].str.replace(',','').str.replace('+','')

In [43]:
#Get unique values except for 'Fewer' and nan
unique_vals = df[~(df['tomatometer_count'] == 'Fewer')]['tomatometer_count'].unique().astype(float)
unique_vals = unique_vals[~np.isnan(unique_vals)]
print("Tomatometer min:",unique_vals.min())

#Get unique values except for 'Fewer' and nan
unique_vals = df[~(df['audience_count'] == 'Fewer')]['audience_count'].unique().astype(float)
unique_vals = unique_vals[~np.isnan(unique_vals)]
print("Audience min:",unique_vals.min())

Tomatometer min: 1.0
Audience min: 50.0


So, we'll set **'Fewer'** for `tomatometer_count = 0` and `audience_count = 25`

In [44]:
#Convert 'fewer'
df.loc[df['tomatometer_count'] == 'Fewer', 'tomatometer_count'] = 0
df.loc[df['audience_count'] == 'Fewer', 'audience_count'] = 25
#Convert to float
df['audience_count'] = df['audience_count'].astype(float)
df['tomatometer_count'] = df['tomatometer_count'].astype(float)

- ```runtime``` will be converted to minutes.

In [45]:
def convert_to_mins(x):
    regex = r'(\d{,1}h)?(\d{,2}m)?'
    r = re.search(regex, x)
    hours = int(r.group(1).strip('h')) if r.group(1) != None else 0
    mins = int(r.group(2).strip('m')) if r.group(2) != None else 0
    result = hours*60 + mins
    return result

df['runtime'] = df['runtime'].str.replace(' ','').apply(convert_to_mins)
#If runtime == 0, the data is wrong, let's convert those to nan
df.loc[df['runtime'] == 0, 'runtime'] = np.nan

### For numerical columns, how are values distributed?
- Percentage of missing values?

In [46]:
df.select_dtypes('number').isna().sum() / len(df)

tomatometer_score    0.043621
tomatometer_count    0.005761
audience_score       0.018930
audience_count       0.018930
runtime              0.004115
dtype: float64

- Describe the values.

In [47]:
df.select_dtypes('number').describe()

,tomatometer_score,tomatometer_count,audience_score,audience_count,runtime
count,1162.000000,1208.000000,1192.000000,1192.000000,1210.000000
mean,0.730077,166.447848,0.742391,41565.247483,112.141322
std,0.238015,130.747810,0.187176,79940.738938,22.256881
min,0.000000,0.000000,0.000000,25.000000,39.000000
25%,0.600000,53.000000,0.640000,250.000000,96.000000
50%,0.810000,143.000000,0.790000,2500.000000,109.000000
75%,0.920000,256.000000,0.890000,25000.000000,124.000000
max,1.000000,602.000000,1.000000,250000.000000,242.000000


All values seem to be normal.

In [48]:
df.head()

,name,genre,tomatometer_score,tomatometer_count,audience_score,audience_count,classification,runtime,release_year,original_language
0,A Castle for Christmas,Holiday,0.74,23.0,0.40,100.0,Not Rated,98.0,2021,English
1,Pinocchio,Kids & family,1.00,61.0,0.73,250000.0,G,87.0,1940,English
2,The Informer,Mystery & thriller,0.64,58.0,0.60,250.0,R,113.0,2019,English
3,They Cloned Tyrone,Sci-fi,0.95,129.0,1.00,25.0,R,122.0,2023,English
4,1917,War,0.89,472.0,0.88,25000.0,R,119.0,2019,English


### Save processed data

In [49]:
df.to_csv('../../data/processed/data.csv', index=False)